# **Importing Dependencies**

In [6]:
import os
import json
from zipfile import ZipFile
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# **Downloading dataset from kaggle**

In [8]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
100%|███████████████████████████████████████| 25.7M/25.7M [00:00<00:00, 265MB/s]
100%|███████████████████████████████████████| 25.7M/25.7M [00:00<00:00, 265MB/s]


In [9]:
with ZipFile("imdb-dataset-of-50k-movie-reviews.zip", "r") as zip_ref:
  zip_ref.extractall()

# **Data Processing**

In [10]:
data = pd.read_csv("IMDB Dataset.csv")

In [11]:
data.shape

(50000, 2)

In [12]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


# **Checking the label distribution**

In [13]:
data['sentiment'].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

**So it is a balanced dataset**

# **Converting catagorical to numerical**

In [14]:
data.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)

<ipython-input-14-c1a779bd745a>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)


In [15]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


# **Train-Test Splitting**

In [16]:
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# **Checking the shape**

In [17]:
print("Shape of train data:", train_data.shape)
print("Shape of test data:", test_data.shape)

Shape of train data: (40000, 2)
Shape of test data: (10000, 2)


# **Lets's Tokenize the data**

In [18]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data["review"])
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]), maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data["review"]), maxlen=200)

In [19]:
print(X_train)

[[1935    1 1200 ...  205  351 3856]
 [   3 1651  595 ...   89  103    9]
 [   0    0    0 ...    2  710   62]
 ...
 [   0    0    0 ... 1641    2  603]
 [   0    0    0 ...  245  103  125]
 [   0    0    0 ...   70   73 2062]]


In [20]:
Y_train = train_data["sentiment"]
Y_test = test_data["sentiment"]

# **Let's build  LSTM model**

In [21]:
model=Sequential()
model.add(Embedding(5000, 128, input_length=200))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


# **Fitting the model**

In [22]:
model.fit(X_train, Y_train, epochs=10, batch_size=64, validation_split=0.2)

Epoch 1/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 111s 209ms/step - accuracy: 0.7177 - loss: 0.5393 - val_accuracy: 0.7881 - val_loss: 0.4530
Epoch 2/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 104s 207ms/step - accuracy: 0.8246 - loss: 0.4100 - val_accuracy: 0.8189 - val_loss: 0.4080
Epoch 3/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 104s 209ms/step - accuracy: 0.8199 - loss: 0.4049 - val_accuracy: 0.8564 - val_loss: 0.3469
Epoch 4/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 104s 208ms/step - accuracy: 0.8602 - loss: 0.3347 - val_accuracy: 0.8599 - val_loss: 0.3426
Epoch 5/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 104s 209ms/step - accuracy: 0.8678 - loss: 0.3178 - val_accuracy: 0.8554 - val_loss: 0.3398
Epoch 6/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 105s 210ms/step - accuracy: 0.8953 - loss: 0.2577 - val_accuracy: 0.8595 - val_loss: 0.3460
Epoch 7/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 104s 208ms/step - accuracy: 0.9131 - loss: 0.2190 - val_accuracy: 0.8815 - val_loss: 0.2927
Epoch 8/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 103s 207ms/step - accuracy: 0.9278 -

# **Evaluating the model**

In [23]:
loss, accuracy = model.evaluate(X_test, Y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 23s 73ms/step - accuracy: 0.8797 - loss: 0.3321
Test Loss: 0.3272348940372467
Test Accuracy: 0.8849999904632568


# **Let's check the model**

In [24]:
def predicting(review):
  sequence = tokenizer.texts_to_sequences([review])
  padded_sequence = pad_sequences(sequence, maxlen=200)
  prediction = model.predict(padded_sequence)
  sentiment = "positive" if prediction[0][0] > 0.5 else "negative"
  return sentiment

In [25]:
review = "This movie was not so interesting."
sentiment = predicting(review)
print(f"The sentiment of the review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 232ms/step
The sentiment of the review is: negative


In [26]:
review = "This movie was very amazing."
sentiment = predicting(review)
print(f"The sentiment of the review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
The sentiment of the review is: positive


In [27]:
review = "This is a boring movie ."
sentiment = predicting(review)
print(f"The sentiment of the review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
The sentiment of the review is: negative


In [28]:
review = "This is a good movie ."
sentiment = predicting(review)
print(f"The sentiment of the review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
The sentiment of the review is: positive


In [29]:
review = "Wow! This is a amazing movie ."
sentiment = predicting(review)
print(f"The sentiment of the review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
The sentiment of the review is: positive


In [30]:
review = "The movie was not good as I expected."
sentiment = predicting(review)
print(f"The sentiment of the review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
The sentiment of the review is: negative


In [31]:
review = "This movie is going to be flop ."
sentiment = predicting(review)
print(f"The sentiment of the review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
The sentiment of the review is: negative


In [32]:
review = "This movie is going to be hit."
sentiment = predicting(review)
print(f"The sentiment of the review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
The sentiment of the review is: positive


In [33]:
import pickle
from tensorflow.keras.models import load_model

# Save the trained model
model.save('sentiment_lstm_model.h5')

# Save the tokenizer using pickle
with open('tokenizer.pkl', 'wb') as file:
    pickle.dump(tokenizer, file)

print("Model and tokenizer saved successfully!")

Model and tokenizer saved successfully!
